In [2]:
# Load packages
import pandas as pd
import numpy as np
import math

In [3]:
# Read the data
df = pd.read_csv(r'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data',header=None,
                 names=['sepal length','sepal width','petal length','petal width','class'])

data = df.iloc[:,:-1].as_matrix()
target = df.iloc[:,-1].as_matrix()
name = df.columns.values

df.head()

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Question 1: 
# Build up a fully grown binary decision tree based on Iris dataset.

In [34]:
# Decision tree.
class Node(object):
    def __init__(self, position="", feature="", split=0, data=[], target=[], left=[], right=[]):
        self.position=position # root, left, or right
        self.feature=feature # Name of the best feature. Use this feature to split current node.
        self.split=split # Best split point of the best feature. 
        self.data=data # Data of features.
        self.target=target # Last column. 
        self.left = left # Left node.
        self.right = right # Right node.

    def __str__(self, level=0):
        ret = ""
        if len(set(self.target))>1:
            ret = "\t"*level+repr(self.position+": "+self.feature+" <= "+str(self.split))+"\n"
        else:
            ret = "\t"*level+repr(self.position+": "+self.target[0])+"\n"
        
        if self.left: ret += self.left.__str__(level+1)
        if self.right: ret += self.right.__str__(level+1)
        return ret

    def __repr__(self):
        return '<tree node representation>'

In [54]:

class DecisionTree(object):
    
    def __init__(self, data, target, method):
        self.tree=Node(position="root",data=data, target=target)
        self.method=method
    
    # Define functions to choose best split.
    # Copy, paste, and modify the codes from lab6.

    # step 1: How to calculate impurity of one node? Please use these two methods: gini and entropy.
    def impurity_node(self,node, method):
        # node: a list of classes of all instances at current node. For example, node = [yes, yes, no, no, yes]
        if len(node)==0:
            return 0

        classes = list(set(node))
        if method == "gini":
            gini = 1
            for i in range(len(classes)):
                p = (node==classes[i]).sum() / float(len(node))
                gini -= p**2
            return gini
        elif method == "entropy":
            entropy = 0
            for i in range(len(classes)):
                p = (node==classes[i]).sum() / float(len(node))
                if p > 0:
                    entropy += -p*math.log(p)
            return entropy
    

    # step 2: How to combine Gini index of two children?
    def impurity_children(self, leftnode, rightnode, method):
        leftgini = self.impurity_node(leftnode, method)
        rightgini = self.impurity_node(rightnode, method)
        leftnum = len(leftnode)
        rightnum = len (rightnode)
        allnum = float(leftnum + rightnum)
        gini = leftnum/allnum * leftgini + rightnum/allnum * rightgini
        return gini

    # step 3: How to calculate value of Gain?
    def gain(self, parent, leftnode, rightnode, method):
        return self.impurity_node(parent, method)-self.impurity_children(leftnode, rightnode, method)

    # step 4: Given data and target, return the index of best feature and the best split point of this feature.
    # For example, return 2 and 1.8 if the best feature is petal length and best split point of petal length is 1.8.
    def bestsplit(self, data, target, method):
        allfeaturegain = []
        allfeaturesplit = []
        for j in range(np.size(data,1)):
            feature_gain = []
            #feature = list(set(data[:,j]))
            feature = data[:,j]
            feature_sorted = sorted(feature)
            splitpoints = [(feature_sorted[i]+feature_sorted[i+1])/2 for i in range(len(feature_sorted)-1)]
            splitpoints.insert(0,feature_sorted[0]-10)
            splitpoints.append(feature_sorted[-1]+10)
            for i in range(len(splitpoints)):
                leftnode = target[feature <= splitpoints[i]]
                rightnode = target[feature > splitpoints[i]]
                feature_gain.append(self.gain(target, leftnode, rightnode, method))
            allfeaturegain.append(max(feature_gain))
            allfeaturesplit.append(splitpoints[feature_gain.index(max(feature_gain))])

        index = allfeaturegain.index(max(allfeaturegain))
        split = allfeaturesplit[index]

        return index, split
    
    # Recursively build the decision tree.
    def nodesplit(self, node, method):
        data = node.data
        target = node.target

        # Call bestsplit() to calculate best feature and best split point.
        index, split = self.bestsplit(data, target, method)

        # Update node.
        node.feature = name[index]
        node.split = split

        # Split data based on best feature and best split point.
        leftindex = data[:,index]<=split
        rightindex = data[:,index]>split
        lefttarget = target[leftindex]
        righttarget = target[rightindex]

        # Recursively call nodesplit() if child node (1) is not empty  and (2) has more than one classes.
        node.left = Node()
        node.left.position="left"
        node.left.data = data[leftindex, :]
        node.left.target = lefttarget
        if len(set(lefttarget)) > 1:    
            self.nodesplit(node.left, method)

        node.right = Node()
        node.right.position="right"
        node.right.data = data[rightindex, :]
        node.right.target = righttarget
        if len(set(righttarget)) > 1:            
            self.nodesplit(node.right, method)
            
    def build(self):
        self.nodesplit(self.tree, self.method)
        print self.tree
        
    def test(self, data, target):
        n = 0
        for sample, sampleclass in zip(data, target):
            tree = self.tree
            while len(set(tree.target)) > 1:
                index = list(name).index(tree.feature)
                if sample[index] <= tree.split:
                    tree = tree.left
                else:
                    tree = tree.right
            n += (tree.target[0] == sampleclass)
        print "Prediction accuracy is", float(n)/len(data)
        print
         
    
print "Build decision tree using gini index:\n"
tree = DecisionTree(data=data, target=target, method="gini")
tree.build()
tree.test(data, target)

print "Build decision tree using entropy:\n"
tree = DecisionTree(data=data, target=target, method="entropy")
tree.build()
tree.test(data, target)


Build decision tree using gini index:

'root: petal length <= 1.9'
	'left: Iris-setosa'
	'right: petal width <= 1.7'
		'left: petal length <= 4.9'
			'left: petal width <= 1.6'
				'left: Iris-versicolor'
				'right: Iris-virginica'
			'right: petal width <= 1.5'
				'left: Iris-virginica'
				'right: sepal length <= 6.95'
					'left: Iris-versicolor'
					'right: Iris-virginica'
		'right: petal length <= 4.8'
			'left: sepal length <= 5.95'
				'left: Iris-versicolor'
				'right: Iris-virginica'
			'right: Iris-virginica'

Prediction accuracy is 1.0

Build decision tree using entropy:

'root: petal length <= 1.9'
	'left: Iris-setosa'
	'right: petal width <= 1.7'
		'left: petal length <= 4.9'
			'left: petal width <= 1.6'
				'left: Iris-versicolor'
				'right: Iris-virginica'
			'right: petal width <= 1.5'
				'left: Iris-virginica'
				'right: sepal length <= 6.95'
					'left: Iris-versicolor'
					'right: Iris-virginica'
		'right: petal length <= 4.8'
			'left: sepal length <= 5.95'